# 1. ket noi sql va tao duong dan
# ket noi moi dung duoc nha

In [27]:
import os
import psycopg2
import pandas as pd
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
from pymilvus import connections, Collection

# ================== Config ==================
DB_PARAMS = {
    "dbname": "video_db",   # database đã tạo
    "user": "postgres",
    "password": "123",      # đổi theo mật khẩu của bạn
    "host": "localhost",
    "port": "5432"
}

CSV_DIR   = "/home/shared/data_batch1/map-keyframes/"  # file CSV mapping cho mỗi video
FRAME_DIR = "/home/shared/data_batch1/Keyframes/"      # thư mục chứa nhiều nhóm + nhiều anh con
VIDEO_DIR = "/home/shared/data_batch1/Videos/"         # thư mục chứa video

MODEL_DIR = "/home/shared/huggingface_cache"  # nơi lưu model CLIP

# ================== Kết nối DB ==================
conn = psycopg2.connect(**DB_PARAMS)
cur = conn.cursor()

# ================== Kết nối Milvus ==================
connections.connect(alias="default", host='localhost', port='19530')

# ================== Load CLIP model CPU ==================
device = "cpu"
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32", cache_dir=MODEL_DIR).to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32", cache_dir=MODEL_DIR)


In [28]:
from pymilvus import connections
print(connections.list_connections())


[('default', <pymilvus.client.grpc_handler.GrpcHandler object at 0x7cee60794a50>)]


In [29]:
from pymilvus import Collection

collection = Collection("text_image_video_collection")
print("Số lượng bản ghi hiện tại:", collection.num_entities)


Số lượng bản ghi hiện tại: 177321


In [13]:
from pymilvus import Collection

# load collection
collection = Collection("text_image_video_collection")

# In schema ra để xem các field
print(collection.schema)

# Hoặc in chi tiết từng field
for field in collection.schema.fields:
    print("Field name:", field.name, "| Type:", field.dtype, "| Desc:", field.description)


{'auto_id': True, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 512}}], 'enable_dynamic_field': True}
Field name: id | Type: 5 | Desc: 
Field name: vector | Type: 101 | Desc: 


# 2. tao conlection (chay 1 lan) 
# khong chay o day

In [ ]:
from pymilvus import MilvusClient
# Kết nối Milvus
milvus_client = MilvusClient(uri="http://localhost:19530")

# Tên collection
collection_name = "text_image_video_collection"

# Xóa collection cũ (nếu có) -> khởi tạo lại từ đầu
# if milvus_client.has_collection(collection_name):
#     milvus_client.drop_collection(collection_name)
#     print(f"⚠️ Collection '{collection_name}' đã bị xóa") # vay cho chac

# Tạo collection mới
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=512,              # Vector size của CLIP ViT-B/32
    auto_id=True,               # Tự động tạo ID
    enable_dynamic_field=True   # Cho phép thêm field động (vd: frame_path)
)

# Kiểm tra danh sách collection hiện có
collections = milvus_client.list_collections()
print("✅ Danh sách collection:", collections) # dung chay, mat het data do

⚠️ Collection 'text_image_video_collection' đã bị xóa
✅ Danh sách collection: ['image_collection', 'text_image_video_collection']


# 3. Load CLIP model (run cai nay khi can query)

In [30]:
import torch
from PIL import Image
from transformers import CLIPModel, CLIPProcessor

# Chỉ dùng CPU
device = torch.device("cpu")

# Tải mô hình CLIP đã huấn luyện trước từ Hugging Face, lưu tại MODEL_DIR
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32", cache_dir=MODEL_DIR).to(device)

# Tải processor của CLIP
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32", cache_dir=MODEL_DIR)

def encode_image(image_path):
    """
    Mã hóa hình ảnh thành vector đặc trưng bằng CLIP trên CPU.
    """
    try:
        image = Image.open(image_path)
        inputs = processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            image_features = model.get_image_features(**inputs)
        return image_features[0].cpu().numpy()
    except Exception as e:
        print(f"Lỗi khi xử lý {image_path}: {e}")
        return None


# Hàm insert tách riêng (chay roi khong can chay nua)

In [19]:
# ================== Hàm insert vào Milvus 3.x ==================
def insert_to_milvus(frame_path, emb):
    """
    Chèn vector đặc trưng và đường dẫn frame vào collection trong Milvus 3.x.

    Args:
        frame_path (str): Đường dẫn tới file frame (hình ảnh).
        emb (numpy.ndarray): Vector đặc trưng của frame (thường từ mô hình như CLIP).

    Returns:
        str: ID của bản ghi vừa chèn trong Milvus, hoặc None nếu lỗi.
    """
    try:
        # Chèn dữ liệu vào collection đã tạo
        res = milvus_client.insert(
            collection_name=collection_name,
            data=[{"vector": emb.tolist(), "frame_path": frame_path}]
        )
        # Trả về ID đầu tiên nếu có
        return res["ids"][0] if "ids" in res and len(res["ids"]) > 0 else None

    except Exception as e:
        print(f"❌ Lỗi insert Milvus cho {frame_path}: {e}")
        return None


# ================== Hàm insert vào PostgreSQL ==================
def insert_to_postgres(video_id, frame_path, pts_time, frame_idx, fps, milvus_id):
    """
    Chèn thông tin frame vào bảng frame_mappings trong PostgreSQL.

    Args:
        video_id (str): ID của video chứa frame.
        frame_path (str): Đường dẫn tới file frame.
        pts_time (float): Thời gian trình bày (presentation timestamp) của frame trong video.
        frame_idx (int): Chỉ số của frame trong video.
        fps (float): Tốc độ khung hình (frames per second) của video.
        milvus_id (str): ID của bản ghi tương ứng trong Milvus.

    Returns:
        None
    """
    try:
        cur.execute("""
            INSERT INTO frame_mappings (video_id, frame_path, pts_time, frame_idx, fps, milvus_id)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON CONFLICT (frame_path) DO NOTHING
        """, (video_id, frame_path, pts_time, frame_idx, fps, milvus_id))
        conn.commit()  # commit ngay sau insert
    except Exception as e:
        print(f"❌ Lỗi insert Postgres cho {frame_path}: {e}")


# Cell 6 – Index toàn bộ video (dùng khi mới chạy lần đầu hoặc re-index)

In [7]:
# ================== Cell 6: Index dữ liệu ==================
def insert_to_milvus(frame_path, emb):
    """
    Chèn vector đặc trưng và đường dẫn frame vào collection Milvus 3.x.
    """
    try:
        res = milvus_client.insert(
            collection_name=collection_name,
            data=[{"vector": emb.tolist(), "frame_path": frame_path}]
        )
        return res["ids"][0] if "ids" in res and len(res["ids"]) > 0 else None
    except Exception as e:
        print(f"❌ Lỗi insert Milvus cho {frame_path}: {e}")
        return None

def insert_to_postgres(video_id, frame_path, pts_time, frame_idx, fps, milvus_id):
    """
    Chèn metadata của frame vào bảng frame_mappings trong PostgreSQL.
    """
    try:
        cur.execute("""
            INSERT INTO frame_mappings (video_id, frame_path, pts_time, frame_idx, fps, milvus_id)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON CONFLICT (frame_path) DO NOTHING
        """, (video_id, frame_path, pts_time, frame_idx, fps, milvus_id))
        conn.commit()
    except Exception as e:
        print(f"❌ Lỗi insert Postgres cho {frame_path}: {e}")

def index_videos():
    """
    Pipeline lập chỉ mục video: mã hóa frame, lưu vector vào Milvus và metadata vào PostgreSQL.
    """
    # Duyệt các thư mục nhóm (keyframes_L21, keyframes_L22,...)
    for group_dir in os.listdir(FRAME_DIR):
        group_path = os.path.join(FRAME_DIR, group_dir)
        if not os.path.isdir(group_path):
            continue

        # Duyệt thư mục video con (L21_V001, L21_V002,...)
        for key_frame_dir in os.listdir(group_path):
            frame_dir_path = os.path.join(group_path, key_frame_dir)
            if not os.path.isdir(frame_dir_path):
                continue

            # CSV và video path
            csv_path = os.path.join(CSV_DIR, f"{key_frame_dir}.csv")
            video_folder_candidates = [f for f in os.listdir(VIDEO_DIR) if group_dir.replace("keyframes_", "videos_") in f]
            if not os.path.exists(csv_path) or not video_folder_candidates:
                print(f"⚠️ Thiếu CSV hoặc video cho {key_frame_dir}, bỏ qua.")
                continue

            video_path = os.path.join(VIDEO_DIR, video_folder_candidates[0], f"{key_frame_dir}.mp4")
            if not os.path.exists(video_path):
                print(f"⚠️ Không tìm thấy file video {video_path}, bỏ qua.")
                continue

            # Đọc CSV
            mapping_df = pd.read_csv(csv_path)

            # Chèn video vào PostgreSQL hoặc lấy video_id
            cur.execute("""
                INSERT INTO videos (video_path, title, description)
                VALUES (%s, %s, %s)
                ON CONFLICT (video_path) DO NOTHING
                RETURNING id
            """, (video_path.replace("\\", "/"), os.path.basename(video_path), "Video demo"))
            row = cur.fetchone()
            if row:
                video_id = row[0]
            else:
                cur.execute("SELECT id FROM videos WHERE video_path=%s", (video_path.replace("\\", "/"),))
                row = cur.fetchone()
                if row:
                    video_id = row[0]
                else:
                    print(f"❌ Không tìm thấy video {video_path}, bỏ qua.")
                    continue

            # Duyệt từng frame trong thư mục video
            for frame_file in os.listdir(frame_dir_path):
                if not frame_file.endswith((".jpg", ".jpeg", ".png")):
                    continue
                frame_path = os.path.join(frame_dir_path, frame_file).replace("\\", "/")
                try:
                    frame_idx = int(os.path.splitext(frame_file)[0])
                except ValueError:
                    continue

                row = mapping_df[mapping_df["n"] == frame_idx]
                if row.empty:
                    continue
                pts_time = float(row["pts_time"].values[0])
                fps = int(row["fps"].values[0])

                # Encode frame
                emb = encode_image(frame_path)
                if emb is None:
                    continue

                # Insert vào Milvus và PostgreSQL
                milvus_id = insert_to_milvus(frame_path, emb)
                if milvus_id is None:
                    continue
                insert_to_postgres(video_id, frame_path, pts_time, frame_idx, fps, milvus_id)

            conn.commit()
            print(f"✅ Indexed {key_frame_dir}")

# ================== Chạy index ==================
index_videos()


✅ Indexed L27_V002
✅ Indexed L27_V010
✅ Indexed L27_V007
✅ Indexed L27_V005
✅ Indexed L27_V006
✅ Indexed L27_V001
✅ Indexed L27_V015
✅ Indexed L27_V012
✅ Indexed L27_V004
✅ Indexed L27_V008
✅ Indexed L27_V014
✅ Indexed L27_V013
✅ Indexed L27_V011
✅ Indexed L27_V016
✅ Indexed L27_V003
✅ Indexed L27_V009
✅ Indexed L29_V011
✅ Indexed L29_V013
✅ Indexed L29_V009
✅ Indexed L29_V008
✅ Indexed L29_V007
✅ Indexed L29_V020
✅ Indexed L29_V003
✅ Indexed L29_V017
✅ Indexed L29_V014
✅ Indexed L29_V016
✅ Indexed L29_V002
✅ Indexed L29_V001
✅ Indexed L29_V012
✅ Indexed L29_V019
✅ Indexed L29_V010
✅ Indexed L29_V023
✅ Indexed L29_V004
✅ Indexed L29_V015
✅ Indexed L29_V018
✅ Indexed L29_V022
✅ Indexed L29_V005
✅ Indexed L29_V021
✅ Indexed L29_V006
✅ Indexed L30_V025
✅ Indexed L30_V033
✅ Indexed L30_V083
✅ Indexed L30_V052
✅ Indexed L30_V054
✅ Indexed L30_V012
✅ Indexed L30_V046
✅ Indexed L30_V040
✅ Indexed L30_V020
✅ Indexed L30_V027
✅ Indexed L30_V005
✅ Indexed L30_V092
✅ Indexed L30_V055
✅ Indexed L3

# fix L26

In [ ]:
from pymilvus import Collection

collection_name = "text_image_video_collection"
collection = Collection(collection_name)  # dùng kết nối mặc định 'default'

def index_videos_incremental():
    """
    Chỉ index các video/frame chưa có trong PostgreSQL / Milvus.
    """
    for group_dir in os.listdir(FRAME_DIR):
        group_path = os.path.join(FRAME_DIR, group_dir)
        if not os.path.isdir(group_path):
            continue

        video_folder_candidates = [f for f in os.listdir(VIDEO_DIR) if group_dir.replace("keyframes_", "videos_") in f]
        if not video_folder_candidates:
            print(f"⚠️ Không tìm thấy thư mục video {group_dir.replace('keyframes_', 'videos_')}, bỏ qua group {group_dir}")
            continue
        video_folder = os.path.join(VIDEO_DIR, video_folder_candidates[0])

        for key_frame_dir in os.listdir(group_path):
            frame_dir_path = os.path.join(group_path, key_frame_dir)
            if not os.path.isdir(frame_dir_path):
                continue

            csv_path = os.path.join(CSV_DIR, f"{key_frame_dir}.csv")
            if not os.path.exists(csv_path):
                print(f"⚠️ Thiếu CSV {csv_path}, bỏ qua {key_frame_dir}")
                continue

            # Lấy video_id nếu đã tồn tại trong DB
            cur.execute("SELECT id FROM videos WHERE video_path=%s", (os.path.join(video_folder, f"{key_frame_dir}.mp4").replace("\\","/"),))
            row = cur.fetchone()
            if row:
                video_id = row[0]
            else:
                # Chèn video mới
                cur.execute("""
                    INSERT INTO videos (video_path, title, description)
                    VALUES (%s, %s, %s)
                    RETURNING id
                """, (os.path.join(video_folder, f"{key_frame_dir}.mp4").replace("\\","/"), key_frame_dir, "Video demo"))
                video_id = cur.fetchone()[0]
                conn.commit()

            mapping_df = pd.read_csv(csv_path)

            for frame_file in os.listdir(frame_dir_path):
                if not frame_file.endswith((".jpg", ".jpeg", ".png")):
                    continue
                frame_path = os.path.join(frame_dir_path, frame_file).replace("\\","/")

                # Kiểm tra nếu frame đã có trong DB
                cur.execute("SELECT 1 FROM frame_mappings WHERE frame_path=%s", (frame_path,))
                if cur.fetchone():
                    continue  # đã index, bỏ qua

                try:
                    frame_idx = int(os.path.splitext(frame_file)[0])
                except ValueError:
                    continue

                row = mapping_df[mapping_df["n"] == frame_idx]
                if row.empty:
                    continue
                pts_time = float(row["pts_time"].values[0])
                fps = int(row["fps"].values[0])

                emb = encode_image(frame_path)
                if emb is None:
                    continue

                # Insert vào Milvus
                try:
                    mr = collection.insert([{"vector": emb.tolist(), "frame_path": frame_path}])
                    milvus_id = mr.primary_keys[0]
                except Exception as e:
                    print(f"❌ Lỗi insert Milvus cho {frame_path}: {e}")
                    continue

                # Insert vào PostgreSQL
                insert_to_postgres(video_id, frame_path, pts_time, frame_idx, fps, milvus_id)

            conn.commit()
            print(f"✅ Indexed {key_frame_dir}")

# Chạy index incremental
index_videos_incremental()


Đã xóa 0 bản ghi từ frame_mappings.
Đã xóa 1 bản ghi từ videos.


In [26]:
from pymilvus import Collection

collection_name = "text_image_video_collection"
collection = Collection(collection_name)  # dùng kết nối mặc định 'default'

def index_videos_incremental():
    """
    Chỉ index các video/frame chưa có trong PostgreSQL / Milvus.
    """
    for group_dir in os.listdir(FRAME_DIR):
        group_path = os.path.join(FRAME_DIR, group_dir)
        if not os.path.isdir(group_path):
            continue

        video_folder_candidates = [f for f in os.listdir(VIDEO_DIR) if group_dir.replace("keyframes_", "videos_") in f]
        if not video_folder_candidates:
            print(f"⚠️ Không tìm thấy thư mục video {group_dir.replace('keyframes_', 'videos_')}, bỏ qua group {group_dir}")
            continue
        video_folder = os.path.join(VIDEO_DIR, video_folder_candidates[0])

        for key_frame_dir in os.listdir(group_path):
            frame_dir_path = os.path.join(group_path, key_frame_dir)
            if not os.path.isdir(frame_dir_path):
                continue

            csv_path = os.path.join(CSV_DIR, f"{key_frame_dir}.csv")
            if not os.path.exists(csv_path):
                print(f"⚠️ Thiếu CSV {csv_path}, bỏ qua {key_frame_dir}")
                continue

            # Lấy video_id nếu đã tồn tại trong DB
            cur.execute("SELECT id FROM videos WHERE video_path=%s", (os.path.join(video_folder, f"{key_frame_dir}.mp4").replace("\\","/"),))
            row = cur.fetchone()
            if row:
                video_id = row[0]
            else:
                # Chèn video mới
                cur.execute("""
                    INSERT INTO videos (video_path, title, description)
                    VALUES (%s, %s, %s)
                    RETURNING id
                """, (os.path.join(video_folder, f"{key_frame_dir}.mp4").replace("\\","/"), key_frame_dir, "Video demo"))
                video_id = cur.fetchone()[0]
                conn.commit()

            mapping_df = pd.read_csv(csv_path)

            for frame_file in os.listdir(frame_dir_path):
                if not frame_file.endswith((".jpg", ".jpeg", ".png")):
                    continue
                frame_path = os.path.join(frame_dir_path, frame_file).replace("\\","/")

                # Kiểm tra nếu frame đã có trong DB
                cur.execute("SELECT 1 FROM frame_mappings WHERE frame_path=%s", (frame_path,))
                if cur.fetchone():
                    continue  # đã index, bỏ qua

                try:
                    frame_idx = int(os.path.splitext(frame_file)[0])
                except ValueError:
                    continue

                row = mapping_df[mapping_df["n"] == frame_idx]
                if row.empty:
                    continue
                pts_time = float(row["pts_time"].values[0])
                fps = int(row["fps"].values[0])

                emb = encode_image(frame_path)
                if emb is None:
                    continue

                # Insert vào Milvus
                try:
                    mr = collection.insert([{"vector": emb.tolist(), "frame_path": frame_path}])
                    milvus_id = mr.primary_keys[0]
                except Exception as e:
                    print(f"❌ Lỗi insert Milvus cho {frame_path}: {e}")
                    continue

                # Insert vào PostgreSQL
                insert_to_postgres(video_id, frame_path, pts_time, frame_idx, fps, milvus_id)

            conn.commit()
            print(f"✅ Indexed {key_frame_dir}")

# Chạy index incremental
index_videos_incremental()


✅ Indexed L27_V002
✅ Indexed L27_V010
✅ Indexed L27_V007
✅ Indexed L27_V005
✅ Indexed L27_V006
✅ Indexed L27_V001
✅ Indexed L27_V015
✅ Indexed L27_V012
✅ Indexed L27_V004
✅ Indexed L27_V008
✅ Indexed L27_V014
✅ Indexed L27_V013
✅ Indexed L27_V011
✅ Indexed L27_V016
✅ Indexed L27_V003
✅ Indexed L27_V009
✅ Indexed L29_V011
✅ Indexed L29_V013
✅ Indexed L29_V009
✅ Indexed L29_V008
✅ Indexed L29_V007
✅ Indexed L29_V020
✅ Indexed L29_V003
✅ Indexed L29_V017
✅ Indexed L29_V014
✅ Indexed L29_V016
✅ Indexed L29_V002
✅ Indexed L29_V001
✅ Indexed L29_V012
✅ Indexed L29_V019
✅ Indexed L29_V010
✅ Indexed L29_V023
✅ Indexed L29_V004
✅ Indexed L29_V015
✅ Indexed L29_V018
✅ Indexed L29_V022
✅ Indexed L29_V005
✅ Indexed L29_V021
✅ Indexed L29_V006
✅ Indexed L30_V025
✅ Indexed L30_V033
✅ Indexed L30_V083
✅ Indexed L30_V052
✅ Indexed L30_V054
✅ Indexed L30_V012
✅ Indexed L30_V046
✅ Indexed L30_V040
✅ Indexed L30_V020
✅ Indexed L30_V027
✅ Indexed L30_V005
✅ Indexed L30_V092
✅ Indexed L30_V055
✅ Indexed L3

# cell 7. nho chay cell 3 truoc

In [31]:
import pandas as pd
import os
import torch
import torch.nn.functional as F
from pymilvus import Collection

# ================== Các hàm hỗ trợ ==================
def time_to_seconds(time_str):
    try:
        minutes, seconds = map(int, time_str.split(':'))
        return minutes * 60 + seconds
    except ValueError:
        print(f"❌ Lỗi định dạng thời gian: {time_str}")
        return None

def get_frame_idx_from_time(video_path, time_start, time_end, csv_dir):
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    csv_path = os.path.join(csv_dir, f"{video_name}.csv")
    if not os.path.exists(csv_path):
        print(f"❌ Không tìm thấy file CSV: {csv_path}")
        return None
    try:
        df = pd.read_csv(csv_path)
        if not all(col in df.columns for col in ['pts_time', 'frame_idx', 'fps']):
            print(f"❌ CSV {csv_path} thiếu cột cần thiết")
            return None
    except Exception as e:
        print(f"❌ Lỗi đọc CSV {csv_path}: {e}")
        return None

    start_sec = time_to_seconds(time_start)
    end_sec = time_to_seconds(time_end)
    if start_sec is None or end_sec is None:
        return None

    target_time = (start_sec + end_sec) / 2
    df['time_diff'] = abs(df['pts_time'] - target_time)
    closest_row = df.loc[df['time_diff'].idxmin()]
    return int(closest_row['frame_idx'])

def format_time(seconds):
    m = int(seconds // 60)
    s = int(seconds % 60)
    return f"{m}:{s:02d}"

def encode_text(text):
    try:
        inputs = clip_processor(
            text=[text],
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=77
        ).to(device)
        with torch.no_grad():
            text_features = clip_model.get_text_features(**inputs)
        return F.normalize(text_features, p=2, dim=1)[0].cpu().numpy()
    except Exception as e:
        print(f"❌ Lỗi encode text: {e}")
        return None

# ================== Hàm chuẩn LSC/VBS ==================
def group_timestamps(frame_list, gap_threshold=15.0, margin=5.0, min_range=5.0):
    if not frame_list:
        return []

    frame_list = sorted(frame_list, key=lambda x: x[1])
    groups = []
    current_group = [frame_list[0]]

    for i in range(1, len(frame_list)):
        prev_frame, prev_time = frame_list[i - 1]
        cur_frame, cur_time = frame_list[i]

        if cur_time - prev_time <= gap_threshold:
            current_group.append((cur_frame, cur_time))
        else:
            groups.append(current_group)
            current_group = [(cur_frame, cur_time)]
    groups.append(current_group)

    result = []
    for group in groups:
        start_time = max(0, group[0][1] - margin)
        end_time = group[-1][1] + margin
        if end_time - start_time < min_range:
            end_time = start_time + min_range
        frame_idxs = [f[0] for f in group]
        result.append((start_time, end_time, frame_idxs))

    return result


# Cell trước phải chạy
def format_search_results(search_results, csv_dir):
    grouped_results = {}
    for res in search_results:
        video_path = res['video_path']
        video_name = os.path.splitext(os.path.basename(video_path))[0]
        if video_path not in grouped_results:
            grouped_results[video_path] = (video_name, [])
        
        for time_start, time_end in res['time_ranges']:
            frame_idx = get_frame_idx_from_time(video_path, time_start, time_end, csv_dir)
            if frame_idx is not None:
                grouped_results[video_path][1].append((frame_idx, time_start, time_end))
    
    return grouped_results



def search_videos_by_text(text_query, top_k=10, gap_threshold=15.0, margin=5.0, min_range=5.0):
    text_emb = encode_text(text_query)
    if text_emb is None:
        return []

    collection_name = "text_image_video_collection"
    collection = Collection(collection_name)

    results = collection.search(
        data=[text_emb.tolist()],
        anns_field="vector",
        param={"metric_type": "COSINE", "params": {"nprobe": 10}},
        limit=top_k,
        output_fields=["id"]
    )

    output = []
    for hit in results[0]:
        entity_id = hit.id
        cur.execute("""
            SELECT v.video_path, v.title, fm.frame_idx, fm.pts_time
            FROM frame_mappings fm
            JOIN videos v ON fm.video_id = v.id
            WHERE fm.milvus_id=%s
        """, (entity_id,))
        row = cur.fetchone()
        if row:
            video_path, title, frame_idx, pts_time = row
            output.append((video_path, title, frame_idx, pts_time))

    grouped_results = {}
    for video_path, title, frame_idx, pts_time in output:
        if video_path not in grouped_results:
            grouped_results[video_path] = {"title": title, "frames": []}
        grouped_results[video_path]["frames"].append((frame_idx, pts_time))

    final_results = []
    for video_path, data in grouped_results.items():
        segments = group_timestamps(
            data["frames"],
            gap_threshold=gap_threshold,
            margin=margin,
            min_range=min_range
        )
        time_ranges = [(format_time(s), format_time(e)) for s, e, _ in segments]
        final_results.append({
            "video_path": video_path,
            "title": data["title"],
            "time_ranges": time_ranges
        })

    return final_results


import os
import pandas as pd

def keyframe_path_from_frame_idx(video_path, frame_idx, csv_dir, frame_root):
    """
    Map frame_idx -> n (theo CSV) rồi tìm file ảnh keyframe tương ứng.
    Trả về path ảnh hoặc None nếu không tìm thấy.
    """
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    group_name = "keyframes_" + video_name.split("_")[0]    # ví dụ: L24_V026 -> keyframes_L24
    csv_path   = os.path.join(csv_dir, f"{video_name}.csv")
    dir_path   = os.path.join(frame_root, group_name, video_name)

    if not os.path.exists(csv_path):
        return None
    try:
        df = pd.read_csv(csv_path)
    except Exception:
        return None

    # Cần cột 'n' và 'frame_idx'
    if not all(c in df.columns for c in ["n", "frame_idx"]):
        return None

    # Tìm hàng có frame_idx gần nhất với frame_idx yêu cầu
    idx = (df["frame_idx"] - int(frame_idx)).abs().idxmin()
    n_val = int(df.loc[idx, "n"])

    # Thử các biến thể tên file: n, zero-pad 3, zero-pad 4, và fallback theo frame_idx
    candidates = []
    for stem in [n_val, f"{n_val:03d}", f"{n_val:04d}", frame_idx, f"{int(frame_idx):03d}", f"{int(frame_idx):04d}"]:
        for ext in ("jpg", "jpeg", "png"):
            candidates.append(os.path.join(dir_path, f"{stem}.{ext}"))

    for p in candidates:
        if os.path.exists(p):
            return p
    return None



# querry

In [36]:
from deep_translator import GoogleTranslator
import os

# ---------------- Demo ----------------
# Văn bản tiếng Việt cần tìm
text_vi = "CHẢ CỐM TẠO HÌNH ÔM TRỌN VỊ NGON"
# Dịch sang tiếng Anh để match với CLIP
text_en = GoogleTranslator(source='vi', target='en').translate(text_vi)

# Tìm kiếm video dựa trên truy vấn văn bản
search_results = search_videos_by_text(text_en, top_k=20, gap_threshold=15.0)

# Chuyển đổi kết quả sang định dạng video_path -> (video_name, list of (frame_idx, time_start, time_end))
results = format_search_results(search_results, csv_dir=CSV_DIR)

# In kết quả
print(f"Kết quả tìm kiếm cho: \"{text_en}\"")
for video_path, (video_name, frames) in results.items():
    print(f"\nVideo: {video_name}")
    print(f"Path: {video_path}")

    for frame_idx, time_start, time_end in frames:
        img_path = keyframe_path_from_frame_idx(
            video_path=video_path,
            frame_idx=frame_idx,
            csv_dir=CSV_DIR,
            frame_root=FRAME_DIR
        )
        img_path_str = img_path if img_path else "⚠️ Không tìm thấy ảnh"
        print(f"Frame Idx: {frame_idx} -- Xuất hiện từ {time_start} đến {time_end} -- Path: {img_path_str}")



Kết quả tìm kiếm cho: "Nuggets create a delicious taste"

Video: L26_V277
Path: /home/shared/data_batch1/Videos/videos_L26_a/L26_V277.mp4
Frame Idx: 5632 -- Xuất hiện từ 3:40 đến 3:52 -- Path: /home/shared/data_batch1/Keyframes/keyframes_L26/L26_V277/129.jpg

Video: L26_V104
Path: /home/shared/data_batch1/Videos/videos_L26_a/L26_V104.mp4
Frame Idx: 5990 -- Xuất hiện từ 3:48 đến 4:10 -- Path: /home/shared/data_batch1/Keyframes/keyframes_L26/L26_V104/120.jpg

Video: L26_V174
Path: /home/shared/data_batch1/Videos/videos_L26_a/L26_V174.mp4
Frame Idx: 5967 -- Xuất hiện từ 3:53 đến 4:05 -- Path: /home/shared/data_batch1/Keyframes/keyframes_L26/L26_V174/140.jpg

Video: L26_V310
Path: /home/shared/data_batch1/Videos/videos_L26_a/L26_V310.mp4
Frame Idx: 4224 -- Xuất hiện từ 2:44 đến 2:54 -- Path: /home/shared/data_batch1/Keyframes/keyframes_L26/L26_V310/095.jpg

Video: L26_V207
Path: /home/shared/data_batch1/Videos/videos_L26_a/L26_V207.mp4
Frame Idx: 6488 -- Xuất hiện từ 4:14 đến 4:26 -- Path: